In [ ]:
## Project Context & Objective

# This notebook supports an operational analytics initiative focused on improving field technician performance. 
# Field operations leaders face challenges with repeat visits, missed appointments, inconsistent technician efficiency, 
# and prolonged job durations — all of which increase operational costs and reduce customer satisfaction.

# The goal of this notebook is to clean and prepare technician job data in a way that preserves metric accuracy while 
# supporting downstream analysis on performance trends, repeat visit drivers, on-time arrival rates, and efficiency benchmarking.

# All data preparation decisions in this notebook are made with business impact, metric reliability, and analytical reuse in mind.

In [ ]:
# In this step, I import the core libraries required for data manipulation, numerical operations, and pattern matching.

# Pandas is used to load and structure the dataset into a DataFrame for analysis.
# NumPy supports numerical operations that will be used during cleaning and feature engineering.
# Regex (re) is imported to standardize inconsistent or messy text fields later in the cleaning process.

# The dataset is then read directly from a CSV file into a Pandas DataFrame. Loading the raw data without transformations ensures that all data quality 
# issues (missing values, formatting inconsistencies, and outliers) are addressed explicitly and intentionally in subsequent steps, rather than being 
# silently altered during ingestion.

In [250]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('/content/technician_performance_multi_region_messy.csv')

In [ ]:
# Real-world operational datasets often contain inconsistent representations of missing data, such as empty strings, placeholder text (e.g., "N/A", "unknown"),
# or mixed casing. Treating these values as legitimate categories would distort downstream analysis and bias summary statistics.

# In this step, I explicitly define a set of known NULL-like values and standardize them to NaN. This ensures that:

# Missing data is handled consistently across all columns

# Pandas’ native missing-value functionality can be leveraged for imputation and analysis

# Subsequent decisions (e.g., whether to impute or exclude records) are based on true data absence, not formatting noise

# This approach avoids silent data corruption and preserves transparency, making later imputation choices both measurable and defensible.

In [251]:
# ---------------------------------------------------------
# 1. Standardize NULL-like values
# ---------------------------------------------------------
null_values = {"", " ", "nan", "none", "n/a", "na", "unknown", "N/A", "None", "Unknown"}

def clean_null(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    return np.nan if x in null_values else x

for col in df.columns:
    df[col] = df[col].apply(clean_null)

In [ ]:

# Before performing any analysis, it is important to ensure that string-based columns are clean and consistent. 
# Leading and trailing whitespace can cause issues such as duplicate categories, incorrect grouping, or failed joins, 
# even when values appear identical to the human eye.

# In this step, all columns are converted to strings and stripped of extra spaces to enforce uniform formatting. 
# This proactive cleaning decision helps prevent subtle data quality issues later in the workflow and ensures 
# that downstream operations—such as filtering, aggregation, or encoding—produce accurate results.

In [252]:
# ---------------------------------------------------------
# 2. Trim whitespace / remove extra spaces
# ---------------------------------------------------------
for col in df.columns:
    df[col] = df[col].astype(str).str.strip()

In [ ]:
# Real-world datasets often contain inconsistent representations of the same categorical values due to human entry errors, abbreviations, typos, 
# or formatting differences. Left unaddressed, these inconsistencies can fragment categories, distort aggregations, and lead to misleading 
# analytical conclusions.

# In this step, I implemented custom normalization functions for key categorical fields—including technician names, cities, regions, job types, time fields,
#  and binary outcome indicators. Each function follows a consistent strategy:
# - Remove punctuation and unnecessary characters
# - Trim whitespace and standardize casing
# - Map known variations and misspellings to a single, canonical value

# For fields such as technician names and locations, standardization ensures accurate grouping, performance evaluation, and reporting. For time-based fields,
# normalizing multiple formats into a consistent representation enables reliable comparisons and downstream time-based analysis. Binary fields (e.g., 
# success and repeat visits) are normalized to enforce consistent logical values.

# This rule-based approach was chosen over automated fuzzy matching to retain full control over how values are consolidated, minimize unintended merges,
# and ensure business-relevant distinctions are preserved. The result is a cleaner, more reliable dataset that supports accurate analysis and decision-making.


In [253]:
# ---------------------------------------------------------
# 3. Fix inconsistent values in each columns
# ---------------------------------------------------------
# Normalizing tech names
def normalize_tech(name):
    if pd.isna(name):
        return np.nan
    name = re.sub(r"[^\w\s]", "", name)  # remove punctuation
    name = name.strip().title()

    # Map variations to standardized names
    mapping = {
        'John S.':'John Smith',
        'J Smith':'John Smith',
        'John S':'John Smith',
        'J. Smith':'John Smith',
        'Jess T.':'Jessica Taylor',
        'J. Taylor':'Jessica Taylor',
        'J Taylor':'Jessica Taylor',
        'Jess T':'Jessica Taylor',
        'Jessica T':'Jessica Taylor',
        'Maria Gomez':'Maria Gomez',
        'Maria G':'Maria Gomez',
        'Mari Gomez':'Maria Gomez',
        'M Gomez':'Maria Gomez',
        'M. Gomez':'Maria Gomez',
        'Liam P.':'Liam Peterson',
        'Liam P':'Liam Peterson',
        'L P':'Liam Peterson'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in name.lower().replace(" ", ""):
            return mapping[key]

    return name

df["tech_name"] = df["tech_name"].apply(normalize_tech)
#df

# Normalizing city names
def normalize_city(city):
    if pd.isna(city):
        return np.nan
    city = re.sub(r"[^\w\s]", "", city)  # remove punctuation
    city = city.strip().title()

    # Map variations to city names
    mapping = {
        'concord':'Concord',
        'charlotte':'Charlotte',
        'huntersvill':'Huntersville'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in city.lower().replace(" ", ""):
            return mapping[key]

    return city

df["city"] = df["city"].apply(normalize_city)
#df

# Normalizing region names
def normalize_region(region):
    if pd.isna(region):
        return np.nan
    region = re.sub(r"[^\w\s]", "", region)  # remove punctuation
    region = region.strip().title()

    # Map variations to region names
    mapping = {
        'east':'east',
        'East':'east',
        'Suth':'south',
        'South':'south',
        'WEST':'west',
        'West':'west',
        'Noth':'north',
        'North':'north'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in region.lower().replace(" ", ""):
            return mapping[key]

    return city

df["region"] = df["region"].apply(normalize_region)
#df

# Normalizing job_type names
def normalize_job(job_type):
    if pd.isna(job_type):
        return np.nan
    job_type = re.sub(r"[^\w\s]", "", job_type)  # remove punctuation
    job_type = job_type.strip().title()

    # Map variations for job_type
    mapping = {
        'install':'install',
        'reapir':'repair',
        'Repair':'repair',
        'Instal':'install',
        'Install':'install',
        'Maintenance':'maintenence',
        'Maint.':'maintenence',
        'Maint':'maintenence'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in job_type.lower().replace(" ", ""):
            return mapping[key]

    return job_type

df["job_type"] = df["job_type"].apply(normalize_job)
#df

# Normalizing scheduled_time
def normalize_time(scheduled_time):
    if pd.isna(scheduled_time):
        return np.nan
    scheduled_time = re.sub(r"[^\w\s]", "", scheduled_time)  # remove punctuation
    scheduled_time = scheduled_time.strip().title()

    # Map variations for scheduled_time
    mapping = {
        '14:22':'2:22pm',
        '12:3 pm':'12:30pm',
        '07:50':'7:50am',
        '09:5 AM':'9:50am',
        '16:40':'4:40pm',
        '8:15 AM':'8:15am',
        '1:15 pm':'1:15pm',
        '900Am':'9:00am',
        '1422':'2:22pm',
        '123 pm':'12:30pm',
        '750Am':'7:50am',
        '0750':'7:50am',
        '095 AM':'9:50am',
        '1640':'4:40pm',
        '815 AM':'8:15am',
        '1115 pm':'1:15pm',
        '900Am':'9:00am',
        '115 Pm':'1:15pm',
        '1230':'12:30pm',
        '222Pm':'2:22pm',
        '0950Am':'9:50am',
        '440Pm':'4:40pm'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in scheduled_time.lower().replace(" ", ""):
            return mapping[key]

    return scheduled_time

df["scheduled_time"] = df["scheduled_time"].apply(normalize_time)
#df

# Normalizing arrival_time
def normalize_time(arrival_time):
    if pd.isna(arrival_time):
        return np.nan
    arrival_time = re.sub(r"[^\w\s]", "", arrival_time)  # remove punctuation
    arrival_time = arrival_time.strip().title()

    # Map variations to arrival_time
    mapping = {
        '14:22':'2:22pm',
        '12:3 pm':'12:30pm',
        '07:50':'7:50am',
        '09:5 AM':'9:50am',
        '16:40':'4:40pm',
        '8:15 AM':'8:15am',
        '1:15 pm':'1:15pm',
        '900Am':'9:00am',
        '1422':'2:22pm',
        '123 pm':'12:30pm',
        '0750':'7:50am',
        '095 AM':'9:50am',
        '1640':'4:40pm',
        '815 AM':'8:15am',
        '1115 pm':'1:15pm',
        '900Am':'9:00am',
        '115 Pm':'1:15pm',
        '1230':'12:30pm',
        '222Pm':'2:22pm',
        '0950Am':'9:50am',
        '440Pm':'4:40pm',
        '950Am':'9:50am',
        '750Am':'7:50am'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in arrival_time.lower().replace(" ", ""):
            return mapping[key]

    return arrival_time

df["arrival_time"] = df["arrival_time"].apply(normalize_time)
#df

# Normalizing success column
def normalize_success(success):
    if pd.isna(success):
        return np.nan
    success = re.sub(r"[^\w\s]", "", success)  # remove punctuation
    success = success.strip().title()

    # Map variations to arrival_time
    mapping = {
        'No':'No',
        'no':'No',
        'Yes':'Yes',
        'yes':'Yes',
        'Y':'Yes',
        'N':'No'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in success.lower().replace(" ", ""):
            return mapping[key]

    return success

df["success"] = df["success"].apply(normalize_success)

# Normalizing repeat_visit column
def normalize_repeat_visit(repeat_visit):
    if pd.isna(repeat_visit):
        return np.nan
    repeat_visit = re.sub(r"[^\w\s]", "", repeat_visit)  # remove punctuation
    repeat_visit = repeat_visit.strip().title()

    # Map variations to repeat_visit
    mapping = {
        'No':'No',
        'no':'No',
        'Yes':'Yes',
        'yes':'Yes',
        'Y':'Yes',
        'N':'No'

    }

    for key in mapping:
        if key.lower().replace(" ", "") in repeat_visit.lower().replace(" ", ""):
            return mapping[key]

    return repeat_visit

df["repeat_visit"] = df["repeat_visit"].apply(normalize_repeat_visit)

In [ ]:
# Date fields frequently contain formatting inconsistencies or invalid values that can break time-based analysis if left untreated. Rather than
# assuming all dates are correctly formatted, this step explicitly validates and corrects date entries to improve data reliability.

# I first attempt to parse each value using pandas’ built-in datetime conversion with strict error handling. If a date fails validation, I then
# apply a targeted rule to detect and correct common data-entry errors—such as swapped month and day values (e.g., an invalid month greater than 12). 
# Only when a value cannot be confidently corrected is it coerced to a missing value.

# This layered approach balances data preservation with accuracy. By correcting recoverable errors while safely handling truly invalid dates, 
# I reduce unnecessary data loss and ensure that downstream analyses—such as trend analysis, scheduling performance, or seasonality—are based on valid
# and trustworthy date information.

In [254]:
# ---------------------------------------------------------
# 4. Fix date formats (including invalid ones)
# ---------------------------------------------------------
def parse_date(val):
    if pd.isna(val):
        return np.nan

    try:
        return pd.to_datetime(val, errors="raise", dayfirst=False)
    except:
        pass

    # Attempt to fix invalid date like "2024-13-10"
    match = re.match(r"(\d{4})-(\d{2})-(\d{2})", str(val))
    if match:
        y, m, d = match.groups()
        if int(m) > 12 and int(d) <= 12:
            return pd.to_datetime(f"{y}-{d}-{m}", errors="coerce")

    return pd.to_datetime(val, errors="coerce")

df["scheduled_date"] = df["scheduled_date"].apply(parse_date)
#df

In [ ]:
# Time-related fields often appear in multiple formats and cannot be reliably compared or analyzed as raw strings. To enable accurate time-based 
# calculations—such as arrival delays, scheduling efficiency, or duration analysis—these fields must be converted into a standardized datetime format.

# In this step, I convert the scheduled and arrival time columns into pandas datetime objects using controlled coercion. Invalid or unparseable values 
# are safely converted to missing values rather than forcing incorrect interpretations. This decision prioritizes analytical integrity by ensuring that 
# only valid time values are used in downstream calculations.

# Standardizing time fields at this stage allows for consistent comparisons, arithmetic operations, and feature engineering while preventing silent 
# errors that could skew performance metrics or operational insights.

In [255]:
# ---------------------------------------------------------
# 5. Parse and standardize time columns
# ---------------------------------------------------------
def parse_time(t):
    if pd.isna(t):
        return np.nan
    return pd.to_datetime(t, errors="coerce").time()

df["scheduled_time_dt"] = pd.to_datetime(df["scheduled_time"], errors="coerce")
df["arrival_time_dt"] = pd.to_datetime(df["arrival_time"], errors="coerce")


/tmp/ipython-input-3126214838.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["scheduled_time_dt"] = pd.to_datetime(df["scheduled_time"], errors="coerce")
/tmp/ipython-input-3126214838.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["arrival_time_dt"] = pd.to_datetime(df["arrival_time"], errors="coerce")


In [ ]:
# Even after initial normalization, minor formatting inconsistencies—such as extra internal spaces or inconsistent capitalization—can persist in
# location fields. These small issues can still lead to duplicate categories and inaccurate grouping in analysis and reporting.

# In this step, I perform a final standardization pass on the city column by collapsing extra spaces and applying consistent title casing. 
# This ensures that city names are uniformly formatted, reducing the risk of fragmented categories and improving the accuracy of location-based 
# aggregations and visualizations.


In [256]:
# ---------------------------------------------------------
# 6. Standardize city names
# ---------------------------------------------------------
df["city"] = df["city"].str.replace("  ", " ").str.title()

In [ ]:
# The parts_used field contains semi-structured text with inconsistent delimiters, quotation marks, and missing-value indicators. Analyzing this field
#  in its raw form would make it difficult to accurately assess part usage frequency, inventory needs, or repair patterns.

# In this step, I transform the raw text into a structured list format by removing extraneous characters, splitting values on common delimiters, 
# and standardizing part names through consistent casing. Explicit null indicators are filtered out to avoid treating missing values as valid parts.

# Converting this column into a clean, list-based representation enables more reliable downstream analysis, such as exploding part usage, counting 
# frequencies, and identifying common repair components. This approach prioritizes analytical flexibility while preserving all meaningful information from 
# the original field.


In [257]:
# ---------------------------------------------------------
# 7. Clean and standardize parts_used
# ---------------------------------------------------------

def clean_parts(p):
    if pd.isna(p):
        return []
    p = p.replace('"', "").replace("'", "")
    parts = re.split(r"[;,]", p)
    parts = [x.strip().title() for x in parts if x.strip() not in null_values]
    return parts

df["parts_list"] = df["parts_used"].apply(clean_parts)

In [ ]:
# This final step prepares the dataset for analysis and modeling by resolving remaining inconsistencies and aligning features with their intended 
# analytical use.

# Unneeded raw columns are dropped once their standardized equivalents are created, reducing redundancy and improving dataset clarity. Binary outcome fields
# are explicitly converted from string labels to boolean values to support logical operations and modeling workflows. Numeric fields are cast to appropriate
# data types, and missing duration values are filled with zero to reflect the absence of recorded work time rather than an unknown measurement.

# Column names are updated to maintain consistency after transformations, ensuring the dataset remains intuitive and easy to interpret. Finally, a targeted
# correction is applied to replace placeholder or invalid technician names with a known value, preventing orphaned records and preserving analytical continuity.

# Together, these decisions finalize a clean, well-structured dataset that is ready for reliable analysis, reporting, and downstream decision-making.


In [258]:
#---------------------------------------------------
# 8. Last fixes
#---------------------------------------------------

# Dropping unneeded columns
df = df.drop(columns=["scheduled_time", "arrival_time","parts_used"])

# Map Yes → True, No → False
cols = ["success", "repeat_visit"]
df[cols] = df[cols].replace({"Yes": True, "No": False})

# Changing column to numeric
df['duration_minutes'] = df['duration_minutes'].astype(float)

# Fill null values with 0 in duration_minutes column
df["duration_minutes"] = df["duration_minutes"].fillna(0)

# Rename column
df.rename(columns={"parts_list": "parts_used"}, inplace=True)

# Adding technician name where name was show 'Nan'
df.loc[df['tech_name'] == 'Nan', 'tech_name'] = 'Sophia Martinez'



/tmp/ipython-input-1886929491.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols] = df[cols].replace({"Yes": True, "No": False})


In [ ]:
# The original dataset did not reflect real-world outcomes. To make the project more realistic, success probabilities are assigned by job type using
# defined target rates. This probabilistic update ensures the `success` column better represents expected operational performance while preserving variability
#  for analysis.


In [259]:
#-------------------------------------------------------------------------------------
# These next few blocks of code is help from ChatGpt to make my dataset more realistic
# ------------------------------------------------------------------------------------


import numpy as np

# Define target success rates
success_rates = {
    'install': 0.80,
    'maintenance': 0.90,
    'repair': 0.85
}

# Update success column probabilistically by job_type
for job_type, rate in success_rates.items():
    mask = df['job_type'] == job_type
    df.loc[mask, 'success'] = np.random.rand(mask.sum()) < rate


In [ ]:
# To reflect realistic operational behavior, repeat visits are probabilistically assigned based on whether the service was successful. If no repeat is 
# needed, the `repeat_reason` field is cleared to maintain data consistency.

In [260]:
# Set repeat_visit based on success
df['repeat_visit'] = False
df.loc[df['success'] == False, 'repeat_visit'] = np.random.rand((df['success'] == False).sum()) < 0.7  # 60–80%
df.loc[df['success'] == True, 'repeat_visit'] = np.random.rand((df['success'] == True).sum()) < 0.10  # 5–15%

# Clear repeat_reason if no repeat visit
df.loc[df['repeat_visit'] == False, 'repeat_reason'] = np.nan


In [ ]:
# Service durations are updated with realistic ranges based on job type to better reflect actual operational times. This ensures the `duration_minutes` column
#  provides plausible values for analysis and modeling.


In [261]:
# Define realistic duration ranges by job_type
duration_ranges = {
    'install': (45, 180),
    'maintenance': (30, 90),
    'repair': (20, 120)
}

for job_type, (min_dur, max_dur) in duration_ranges.items():
    mask = df['job_type'] == job_type
    df.loc[mask, 'duration_minutes'] = np.random.randint(min_dur, max_dur+1, mask.sum())


In [ ]:
# To make the dataset more realistic, success and repeat visits are adjusted by technician skill. Each technician is assigned a performance profile, 
# reflecting stronger or weaker success rates and likelihood of repeat visits after failure.


In [262]:
techs = ['Jessica Taylor', 'John Smith', 'Nan', 'Liam Peterson', 'Maria Gomez']
df['tech_name'] = np.random.choice(techs, size=len(df))

# Assign performance tiers
performance = {
    'Jessica Taylor': {'success': 0.95, 'repeat_prob_fail': 0.3},  # Strong tech
    'John Smith': {'success': 0.60, 'repeat_prob_fail': 0.9},  # Weak tech
    'Nan': {'success': 0.85, 'repeat_prob_fail': 0.5},  # Average
    'Liam Peterson': {'success': 0.85, 'repeat_prob_fail': 0.5},
    'Maria Gomez': {'success': 0.85, 'repeat_prob_fail': 0.5}
}

# Update success and repeat_visit based on tech
for tech, params in performance.items():
    mask = df['tech_name'] == tech
    df.loc[mask, 'success'] = np.random.rand(mask.sum()) < params['success']
    fail_mask = mask & (df['success'] == False)
    df.loc[fail_mask, 'repeat_visit'] = np.random.rand(fail_mask.sum()) < params['repeat_prob_fail']


In [ ]:
# To simulate real-world conditions, arrival times are adjusted with normally distributed noise around the scheduled time. This creates realistic 
# deviations while keeping extreme values within ±60 minutes.


In [263]:
# Ensure timestamps are datetime
df['scheduled_time_dt'] = pd.to_datetime(df['scheduled_time_dt'])
df['arrival_time_dt'] = pd.to_datetime(df['arrival_time_dt'])

# Add normal noise (mean +7 min, std 15 min), cap ±60 min
noise = np.random.normal(loc=7, scale=15, size=len(df))
noise = np.clip(noise, -60, 60)
df['arrival_time_dt'] = df['scheduled_time_dt'] + pd.to_timedelta(noise, unit='m')


In [ ]:
# To reflect realistic customer behavior, existing customer IDs are randomly assigned to ~85% of rows, leaving some entries as NaN to represent first-time 
# customers. This balances repeat and new customer scenarios for analysis.


In [264]:
# Assume existing customer IDs: df['customer_id']
existing_customers = df['customer_id'].dropna().unique()

# Assign customers to ~85% of rows
mask = np.random.rand(len(df)) < 0.85
df.loc[mask, 'customer_id'] = np.random.choice(existing_customers, size=mask.sum())

# Optional: leave some NaNs for first-time customers


In [ ]:
# The fully cleaned and realistically adjusted dataset is saved to a CSV file for analysis or modeling. This ensures all transformations are preserved
#  and the data is ready for downstream use.


In [265]:
# ---------------------------------------------------------
# Save cleaned output
# ---------------------------------------------------------
df.to_csv("tech_performance_cleaned.csv", index=False)

print("Cleaning complete! Saved as tech_performance_cleaned.csv")

Cleaning complete! Saved as tech_performance_cleaned.csv
